In [2]:
#!pip install psycopg2

import psycopg2
import time
import boto3
import copy
import gc
import pandas as pd

bucket = 'mads-capstone-2023' 
data_key = 'greeks3.csv' 

s3_client = boto3.client('s3')
obj = s3_client.get_object(Bucket=bucket, Key=data_key)

cnt = 0
for i, row in enumerate(obj["Body"].iter_lines()):
    if i==0:
        continue
    cnt += 1
    if (i%10_000_000)==0:
        print(f"{i} Records")

In [2]:
total_records = 196059840

## Connect to DB

In [3]:
db_connection = psycopg2.connect(
            user='postgres', password='Data2023',
            host='mads-capstone.cmohac77hep9.eu-north-1.rds.amazonaws.com', port=5432,
            database="mads"
        )

db_connection.autocommit = True

cursor = db_connection.cursor()

In [12]:
#delete_query = "DROP TABLE greeks4"

#cursor.execute(delete_query)

In [13]:
query = '''CREATE TABLE greeks3 (id SERIAL PRIMARY KEY,  
                                exec_date date,
                                increment SMALLINT,
                                opt_value float,
                                ticker VARCHAR(8),
                                greek VARCHAR(16),
                                option_type VARCHAR(8))'''


cursor.execute(query)

In [14]:
import pandas as pd

select_query = "SELECT count(*) FROM greeks3"

cursor.execute(select_query)

cursor.fetchall()

[(0,)]

In [15]:
%%time

bucket = 'mads-capstone-2023' 
data_key = 'greeks3.csv' 

s3_client = boto3.client('s3')
obj = s3_client.get_object(Bucket=bucket, Key=data_key)

records = []

for i, row in enumerate(obj["Body"].iter_lines()):
    if i == 0:
        continue
    #if i<=3000000: ## First Run failed at # Mn records. Condition to skip records till last fail count.
    #    if (i%1_000_000) ==0:
    #        print(f"{i} Records Passed.")
    #    continue
    try:
        row_data = row.decode().split(",")
        dt, incr, opt_value, ticker, greek, opt_type = row_data
        dt = dt.split()[0]
        incr = int(incr) if incr else None
        opt_value = float(opt_value) if opt_value else None
    except Exception as e:
        print(e)
        print(f"{i} - {row_data} Failed Parsing.")
        continue

    records.append([dt,incr,opt_value,ticker,greek,opt_type])

    if (i%100_000)==0: ## Batching 100k records and uploading all at once.
        df = pd.DataFrame(records)#.dropna()
        df.to_csv("sample1.csv", header=False, index=False)
        with open("sample1.csv", "rb") as fp:
            cursor.copy_from(fp, "greeks3", sep=",", null="", columns=["exec_date", "increment", "opt_value", "ticker", "greek", "option_type"])

        records = []
        gc.collect();

        if (i%1_000_000) == 0:
            print(f"{i} Records Inserted")

if records:
    print(len(records))
    df = pd.DataFrame(records)#.dropna()
    df.to_csv("sample1.csv", header=False, index=False)
    with open("sample1.csv", "rb") as fp:
        cursor.copy_from(fp, "greeks3", sep=",", null="", columns=["exec_date", "increment", "opt_value", "ticker", "greek", "option_type"])

1000000 Records Inserted
2000000 Records Inserted
3000000 Records Inserted
4000000 Records Inserted
5000000 Records Inserted
6000000 Records Inserted
7000000 Records Inserted
8000000 Records Inserted
9000000 Records Inserted
10000000 Records Inserted
11000000 Records Inserted
12000000 Records Inserted
13000000 Records Inserted
14000000 Records Inserted
15000000 Records Inserted
16000000 Records Inserted
17000000 Records Inserted
18000000 Records Inserted
19000000 Records Inserted
20000000 Records Inserted
21000000 Records Inserted
22000000 Records Inserted
23000000 Records Inserted
24000000 Records Inserted
25000000 Records Inserted
26000000 Records Inserted
27000000 Records Inserted
28000000 Records Inserted
29000000 Records Inserted
30000000 Records Inserted
31000000 Records Inserted
32000000 Records Inserted
33000000 Records Inserted
34000000 Records Inserted
35000000 Records Inserted
36000000 Records Inserted
37000000 Records Inserted
38000000 Records Inserted
39000000 Records Inse

In [16]:
import pandas as pd

select_query = "SELECT count(*) FROM greeks3"

cursor.execute(select_query)

cursor.fetchall()

[(196059840,)]

In [8]:
index_query = "CREATE INDEX greeks_ticker ON greeks3(ticker);"

cursor.execute(index_query)

In [4]:
index_query = "CREATE INDEX greeks_greek ON greeks3(greek);"

cursor.execute(index_query)

In [5]:
index_query = "CREATE INDEX greeks_option_type ON greeks3(option_type);"

cursor.execute(index_query)

In [12]:
db_connection.commit()

In [13]:
cursor.close()
db_connection.close()